In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB2 #260x260
from tensorflow.keras.applications.efficientnet import preprocess_input
import imgaug.augmenters as iaa
import imgaug

from sklearn.model_selection import train_test_split

In [26]:
data_dir = '/run/user/1000/gvfs/smb-share:server=titan.local,share=datasets/kitchenware-classification'
seed = 42

In [27]:
df_train_full = pd.read_csv(data_dir+'/train.csv', dtype={'Id': str})
df_extra = pd.read_csv(data_dir+'/data.csv', dtype={'Id': str})
df_train_full = pd.concat([df_train_full, df_extra], ignore_index = True)
df_train_full['filename'] = data_dir+'/images/' + df_train_full['Id'] + '.jpg'
df_train_full.head()

,Id,label,filename
0,0560,glass,/run/user/1000/gvfs/smb-share:server=titan.loc...
1,4675,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...
2,0875,glass,/run/user/1000/gvfs/smb-share:server=titan.loc...
3,4436,spoon,/run/user/1000/gvfs/smb-share:server=titan.loc...
4,8265,plate,/run/user/1000/gvfs/smb-share:server=titan.loc...


In [28]:
df_train_full['label']

0       glass
1         cup
2       glass
3       spoon
4       plate
        ...  
7983    plate
7984    knife
7985    glass
7986    glass
7987      cup
Name: label, Length: 7988, dtype: object

In [29]:
image_width = 260
image_height = 260

In [30]:
df_train, df_val, y_train, y_val = train_test_split(df_train_full, df_train_full['label'], test_size = 0.2, shuffle = True, stratify = df_train_full['label'], random_state = seed)
df_train

,Id,label,filename
7592,eu.95a129ef-d94a-4ab0-a4e4-e66aac61e55f,fork,/run/user/1000/gvfs/smb-share:server=titan.loc...
7704,eu.b90f170d-c2d2-47bf-bd61-375f9032e98f,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...
7840,eu.84a95cb4-6026-40ed-bfb6-936d297a7d91,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...
5030,5485,fork,/run/user/1000/gvfs/smb-share:server=titan.loc...
7114,eu.edfa5512-6f5d-4cf7-bbcc-a31eace9b484,spoon,/run/user/1000/gvfs/smb-share:server=titan.loc...
...,...,...,...
5033,5725,plate,/run/user/1000/gvfs/smb-share:server=titan.loc...
4172,5120,plate,/run/user/1000/gvfs/smb-share:server=titan.loc...
4507,9013,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...
3625,2070,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...


In [31]:
df_val

,Id,label,filename
3699,1024,spoon,/run/user/1000/gvfs/smb-share:server=titan.loc...
5584,eu.f501ebbc-40fa-42e0-8a6c-21e9f0b6f9a7,spoon,/run/user/1000/gvfs/smb-share:server=titan.loc...
2308,6663,knife,/run/user/1000/gvfs/smb-share:server=titan.loc...
3550,1530,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...
4598,7426,fork,/run/user/1000/gvfs/smb-share:server=titan.loc...
...,...,...,...
2764,5270,spoon,/run/user/1000/gvfs/smb-share:server=titan.loc...
5395,2380,fork,/run/user/1000/gvfs/smb-share:server=titan.loc...
2878,2094,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...
2906,2097,knife,/run/user/1000/gvfs/smb-share:server=titan.loc...


In [32]:
train_aug = iaa.Sequential(
    [
        iaa.Resize({"height": image_height, "width": image_width}, interpolation=imgaug.ALL),
        iaa.Fliplr(0.3),
        iaa.Flipud(0.2), # vertically flip 20% of all images
        # Make some images brighter and some darker.
        # In 20% of all cases, we sample the multiplier once per channel,
        # which can end up changing the color of the images.
        iaa.Sometimes(0.3,iaa.Multiply((0.8, 1.2), per_channel=0.2)),
        # `Sometimes()` applies a function randomly to the inputs with
        # a given probability (0.3, in this case).
        iaa.Sometimes(0.3, iaa.Cutout()),
        iaa.Sometimes(0.1, iaa.GaussianBlur(sigma=(0, 0.5))),
        iaa.Sometimes(0.1, iaa.Cartoon(blur_ksize=3, segmentation_size=1.0, saturation=2.0, edge_prevalence=1.0))
    ]
)

In [33]:
train_datagen = ImageDataGenerator(preprocessing_function=train_aug)

In [34]:
train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='filename',
    y_col='label',
    featurewise_center=True,
    featurewise_std_normalization=True,
    batch_size=32,
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_generator = val_datagen.flow_from_dataframe(
    df_val,
    x_col='filename',
    y_col='label',
    featurewise_center=True,
    featurewise_std_normalization=True,

    batch_size=32,
)

Found 6390 validated image filenames belonging to 6 classes.
Found 1598 validated image filenames belonging to 6 classes.


In [35]:
base_model = EfficientNetB2(
    weights='imagenet',
    include_top=False,
    input_shape=(image_width, image_height, 3)
)
base_model.trainable = False

inputs = keras.Input(shape=(image_width, image_height, 3))

base = base_model(inputs, training=False)
vectors = keras.layers.GlobalAveragePooling2D()(base)
outputs = keras.layers.Dense(6)(vectors)

model = keras.Model(inputs, outputs)

2023-01-12 14:27:04.972601: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-12 14:27:05.060234: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-12 14:27:05.060447: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-12 14:27:05.061516: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

31790344/31790344 [==============================] - 1s 0us/step


In [36]:
learning_rate = 0.01
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

loss = keras.losses.CategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [37]:
history = model.fit(
    train_generator,
    epochs=2,
    validation_data=val_generator
)

AssertionError: Expected boolean as argument for 'return_batch', got type <class 'numpy.ndarray'>. Call augment() only with named arguments, e.g. augment(images=<array>).